Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Object detection on IPU using YOLO v4 - inference
This notebook demonstrates an object detection task with a YOLO v4 model using an inference pipeline run on Graphcore IPUs. Originally the code of the YOLO v4 model adapted for IPU was published in the [examples GitHub repository](https://github.com/graphcore/examples/tree/master/vision/yolo_v4/pytorch).

### Summary table

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Vision  | Object detection | YOLO v4 | COCO | Inference | Recommended: POD4 | 2 minutes    |


![object detection on IPU](notebook/first_example.png "Last supper object detection")


## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you. To run the demo using other IPU hardware, you need to have the Poplar SDK enabled and the relevant PopTorch wheels installed. Refer to the [getting started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK and install the PopTorch wheels.


## Requirements
Before using the model on IPU you have to build the custom operations for IPU:

In [ ]:
!make

and install the Python dependencies:

In [ ]:
%pip install -r requirements.txt
from examples_utils import notebook_logging

# TODO: uncomment logger load when issue with permissions is fixed
# %load_ext gc_logger

In order to improve usability and support for future users, Graphcore would like to collect information about the applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext gc_logger` from any cell.

## COCO dataset

This demo of inference with YOLO v4 on IPU uses the checkpoint from a model trained with the [COCO dataset](https://cocodataset.org/). With this we can demonstrate the detection of 80 different classes.

In [ ]:
from ruamel import yaml

class_names = yaml.safe_load(open("configs/class_name.yaml"))["class_names"]
class_names

## Model preparation

In [ ]:
import os
import time
import torch

from PIL import Image
from pathlib import Path

import poptorch

path_to_detection = Path().parent.resolve()
os.environ["PYTORCH_APPS_DETECTION_PATH"] = str(path_to_detection)

We are using the original YOLO v4 model with a checkpoint trained with the COCO dataset, which we pass as `checkpoint`.

In [ ]:
checkpoint = "checkpoint/yolov4_p5_reference_weights/yolov4-p5-sd.pt"

Next we download the checkpoint for the model:

In [ ]:
%%bash

FILE=./checkpoint/yolov4_p5_reference_weights/yolov4-p5-sd.pt; \
if [ -f "$FILE" ]; then \
    echo "$FILE exists, no need to download."; \
else \
    mkdir checkpoint; \
    cd checkpoint; \
    curl https://gc-demo-resources.s3.us-west-1.amazonaws.com/yolov4_p5_reference_weights.tar.gz -o yolov4_p5_reference_weights.tar.gz && tar -zxvf yolov4_p5_reference_weights.tar.gz && rm yolov4_p5_reference_weights.tar.gz; \
    cd ..; \
fi 

We are using a `YOLOv4InferencePipeline` class to set all the IPU specific options and wrap the PyTorch model in a PopTorch inference model. The pipeline reads the configuration parameters from the [config file](configs/inference-yolov4p5.yaml).

In [ ]:
from api import YOLOv4InferencePipeline

pipeline = YOLOv4InferencePipeline(
    checkpoint_path="checkpoint/yolov4_p5_reference_weights/yolov4-p5-sd.pt",
)

We are using the image of a famous painting (The Last Supper by Leonardo da Vinci) to demonstrate how to use the inference model. This image is stored in the repo:

In [ ]:
image = Image.open("notebook/last_supper_restored.jpeg")

The pipeline call carries out the following steps: image preprocessing; a forward pass of the preprocessed image through the model; model output postprocessing.
When used for the first time, a one-time compilation of the model is triggered.
For a single IPU it should take about 3 minutes to compile the model.

In [ ]:
%%time
processed_batch = pipeline(image)

Subsequent calls are much faster:

In [ ]:
%%time
processed_batch = pipeline(image)

You can now check what has been detected on the input image.
The output contains a list of detected objects for each image in the batch.
Each result contains five numbers: the first four are the coordinates in XYWH format (x, y, width, and height of bounding box), the fifth number represents the confidence of the prediction, and the sixth number represents the predicted class of the detected object.

In [ ]:
processed_batch

You can use the coordinates from the of the detected objects to plot bounding boxes on the original image.

In [ ]:
img_paths = pipeline.plot_img(processed_batch, image)
img_paths

## Try it on your own

Let's check the YOLOv4 model on IPU with some images from the Internet.

We can use the `wget` command to store the image as `image.png`. The syntax is `!wget -O image.png [YOUR URL]`, for example:

In [ ]:
# UPLOAD YOUR OWN IMAGE HERE BY REPLACING THE URL

!wget -O image.png https://www.graphcore.ai/hubfs/assets/images/content/new-team.jpg

Let's now use this image for inference with the same model.

In [ ]:
%%time
image = Image.open("image.png")

processed_batch = pipeline(image)
img_paths = pipeline.plot_img(processed_batch, image)

### Optional - Release IPUs in use

The IPython kernel has a lock on the IPUs used in running the model, preventing other users from using them. For example, if you wish to use other notebooks after working your way through this one, it may be necessary to manually run the below cell to release IPUs from use. This will happen by default if using the Run All option. More information on the topic can be found at [Managing IPU Resources](https://github.com/gradient-ai/Graphcore-HuggingFace/blob/main/useful-tips/managing_ipu_resources.ipynb).

In [ ]:
pipeline.detach()